# Codebase Chat Agent

This agent is responsible for answering questions about the codebases. This is a simple ReAct agent with access to a tool to search code snippets.

If the agent need to think about the user query or make function calls, it will do in a tag `<thinking>`. This can used to understand the agent's thinking process. Integrations like OpenWebUI can use this tag to show the agent's thinking process to the user.

## Setup

In [1]:
import asyncio
import os

from rich import print  # noqa: A004

from automation.agents.codebase_chat import CodebaseChatAgent

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

codebase_chat = CodebaseChatAgent()
codebase_chat_agent = await codebase_chat.agent

## Graph

In [ ]:
from IPython.display import Image, display  # noqa: A004

display(Image(await codebase_chat.draw_mermaid()))

## Examples

Here are some examples of how the agent works. 

### Query unrelated to codebases

The expected behavior is that the agent will answer that it can only answer questions about the codebases or software development in general.

In [ ]:
result = await codebase_chat_agent.ainvoke({"messages": [("human", "Hi, what is the capital of France?")]})

for message in result["messages"]:
    print(message.pretty_print())

### Query about software development

The expected behavior is that the agent will answer directly without searching the codebases. 

In [ ]:
result = await codebase_chat_agent.ainvoke({"messages": [("human", "How can I install dependencies?")]})

for message in result["messages"]:
    print(message.pretty_print())

### Query about a specific repository

The expected behavior is that the agent will search the codebases for the query and answer the question based on the codebases.

In [ ]:
questions = [
    "How many agents are there in daiv and what are they?",
    "How can i setup a test project on local GitLab to be used with DAIV?",
    "what are the configuration options for the codebase chat agent?",
    "What are the supported models in DAIV?",
    "Is there a way to configure embeddings for the codebase? If yes, what are the options?",
]
results = await asyncio.gather(*[
    codebase_chat.agent.ainvoke({"messages": [("human", question)]}) for question in questions
])

for result in results:
    for message in result["messages"]:
        message.pretty_print()

### Query about a specific repository with follow-up questions

It's expected that the agent will answer being aware of previous messages.

In [ ]:
result = await codebase_chat_agent.ainvoke({
    "messages": [("human", "What's the python version used on django-webhooks?")]
})
result = await codebase_chat_agent.ainvoke({
    "messages": result["messages"] + [("human", "How can i install a new dependency?")]
})
for message in result["messages"]:
    message.pretty_print()